# Multiscale MILP (using Attributes)

This is a continuation of 'The Scheduling Example', refer to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model()
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [2]:
m.solar = Resource(consume_max=[100] * 4)
m.wind = Resource(consume_max=100 * 4)
m.power = Resource(demand_nominal=180, demand_min=[0.6, 0.7, 0.8, 0.3])

⚖   Initiated solar balance in (l0, q)                                      ⏱ 0.0002 s
🔗  Bound [≤] solar consume in (l0, q)                                       ⏱ 0.0010 s
⚖   Initiated wind balance in (l0, y)                                       ⏱ 0.0001 s
🔗  Bound [≤] wind consume in (l0, y)                                        ⏱ 0.0005 s
⚖   Initiated power balance in (l0, q)                                      ⏱ 0.0001 s
🔗  Bound [≥] power release in (l0, q)                                       ⏱ 0.0009 s


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [3]:
m.wf = Process(
    m.power == -1.0 * m.wind,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    operate_normalize=True,
    operate_max=[0.9, 0.8, 0.5, 0.7],
    capex=990637 + 3354,
    opex=49,
)

🔗  Bound [≤] wf capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≥] wf capacity in (l0, y)                                         ⏱ 0.0001 s
🔗  Bound [≤] wf operate in (l0, q)                                          ⏱ 0.0002 s
🧭  Mapped time for operate (wf, l0, q) ⟺ (wf, l0, y)                        ⏱ 0.0001 s


Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [4]:
m.pv = Process(
    m.power == -1 * m.solar,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    operate_normalize=True,
    operate_max=[0.6, 0.8, 0.9, 0.7],
    capex=567000 + 872046,
    opex=90000,
)

🔗  Bound [≤] pv capacity in (l0, y)                                         ⏱ 0.0004 s
🔗  Bound [≥] pv capacity in (l0, y)                                         ⏱ 0.0002 s
🔗  Bound [≤] pv operate in (l0, q)                                          ⏱ 0.0003 s
🧭  Mapped time for operate (pv, l0, q) ⟺ (pv, l0, y)                        ⏱ 0.0001 s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [5]:
m.lii = Storage(
    m.power == 0.9,
    capacity_max=100,
    capacity_min=10,
    capacity_optional=True,
    charge_capacity_max=100,
    discharge_capacity_max=100,
    capex=1302182 + 41432,
    inventory_cost=2000,
)

🔗  Bound [≤] lii.charge capacity in (l0, y)                                 ⏱ 0.0002 s
🔗  Bound [≤] lii.discharge capacity in (l0, y)                              ⏱ 0.0002 s
🔗  Bound [≤] lii.stored invcapacity in (l0, y)                              ⏱ 0.0002 s
🔗  Bound [≥] lii.stored invcapacity in (l0, y)                              ⏱ 0.0002 s


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [6]:
m.locate(m.wf, m.pv, m.lii)

⚖   Updated power balance with produce(power, l0, q, operate, wf)           ⏱ 0.0001 s
⚖   Updated wind balance with expend(wind, l0, y, operate, wf)              ⏱ 0.0001 s
🏭  Operating streams introduced for wf in l0                                ⏱ 0.0017 s
🏗   Construction streams introduced for wf in l0                            ⏱ 0.0000 s
🌍  Located wf in l0                                                         ⏱ 0.0024 s
⚖   Updated power balance with produce(power, l0, q, operate, pv)           ⏱ 0.0001 s
⚖   Updated solar balance with expend(solar, l0, q, operate, pv)            ⏱ 0.0001 s
🏭  Operating streams introduced for pv in l0                                ⏱ 0.0018 s
🏗   Construction streams introduced for pv in l0                            ⏱ 0.0000 s
🌍  Located pv in l0                                                         ⏱ 0.0022 s
💡  Assumed lii capacity unbounded in (l0, y)                                ⏱ 0.0001 s
🧭  Mapped time for inventory (lii.stored, l

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

## Optimize!

In [7]:
m.usd.spend.opt()

🧭  Mapped samples for spend (usd, l0, y, capacity, wf) ⟺ (usd, l0, y)       ⏱ 0.0001 s
🧭  Mapped samples for spend (usd, l0, y, operate, wf) ⟺ (usd, l0, y)        ⏱ 0.0002 s
🧭  Mapped samples for spend (usd, l0, y, capacity, pv) ⟺ (usd, l0, y)       ⏱ 0.0001 s
🧭  Mapped samples for spend (usd, l0, y, operate, pv) ⟺ (usd, l0, y)        ⏱ 0.0001 s
🧭  Mapped samples for spend (usd, l0, y, invcapacity, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0001 s
🧭  Mapped samples for spend (usd, l0, y, inventory, lii.stored) ⟺ (usd, l0, y) ⏱ 0.0001 s
📝  Generated Program(m).mps                                                 ⏱ 0.0046 s


Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(m).mps
Reading time = 0.00 seconds
PROGRAM(M): 85 rows, 78 columns, 198 nonzeros


📝  Generated gurobipy model. See .formulation                               ⏱ 0.0086 s


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x239d525a
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

📝  Generated Solution object for Program(m). See .solution                  ⏱ 0.0003 s
✅  Program(m) optimized using gurobi. Display using .output()               ⏱ 0.0202 s


In [8]:
m.show(True)

# Mathematical Program for Program(m)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [9]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [10]:
m.produce(m.lii.stored, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [11]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [12]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [13]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
m.capacity.output(aslist=True)

[100.0, 100.0, 100.0, 100.0]